In [3]:
import sys
import os
import math

#from progressbar import ProgressBar, Bar, ETA, Percentage, SimpleProgress
#from ipyparallel import Client

# parallelization
#client = Client(profile='default')

import ROOT
ROOT.gROOT.SetBatch(True)

from DevTools.Analyzer.utilities import deltaR, deltaPhi

# load FWLite C++ libraries
ROOT.gSystem.Load("libFWCoreFWLite.so");
ROOT.gSystem.Load("libDataFormatsFWLite.so");
ROOT.AutoLibraryLoader.enable()

# load FWlite python libraries
from DataFormats.FWLite import Handle, Events

# plotting
import DevTools.Plotter.CMS_lumi as CMS_lumi
import DevTools.Plotter.tdrstyle as tdrstyle
tdrstyle.setTDRStyle()
ROOT.gStyle.SetPalette(1)

import rootnotes
canvas = rootnotes.default_canvas()

In [8]:

DeltaR = lambda a,b: deltaR(a.eta(),a.phi(),b.eta(),b.phi())
DeltaPhi = lambda a,b: deltaPhi(a.phi(),b.phi())

def process(events,func,func_args=[],func_kwargs={},**kwargs):
    maxEvents = kwargs.pop('maxEvents',-1)
    reportEvery = kwargs.pop('reportEvery',1000)
    
    muons, muonLabel = Handle("std::vector<pat::Muon>"), "slimmedMuons"
    electrons, electronLabel = Handle("std::vector<pat::Electron>"), "slimmedElectrons"
    photons, photonLabel = Handle("std::vector<pat::Photon>"), "slimmedPhotons"
    taus, tauLabel = Handle("std::vector<pat::Tau>"), "slimmedTaus"
    jets, jetLabel = Handle("std::vector<pat::Jet>"), "slimmedJets"
    mets, metLabel = Handle("std::vector<pat::MET>"), "slimmedMETs"
    vertices, vertexLabel = Handle("std::vector<reco::Vertex>"), "offlineSlimmedPrimaryVertices"
    beamspot, beamspotLabel = Handle("reco::BeamSpot"), "offlineBeamSpot"
    genParticles, genParticleLabel = Handle("std::vector<reco::GenParticle>"), "prunedGenParticles"
    
    for i,event in enumerate(events):
        if maxEvents>=0 and i>maxEvents: break
        if i%reportEvery==1: print 'Processing event {0}'.format(i)
            
        event.getByLabel(muonLabel, muons) 
        event.getByLabel(electronLabel, electrons)
        event.getByLabel(photonLabel, photons)
        event.getByLabel(tauLabel, taus)
        event.getByLabel(jetLabel, jets)
        event.getByLabel(metLabel, mets)
        event.getByLabel(vertexLabel, vertices)
        event.getByLabel(beamspotLabel,beamspot)
        event.getByLabel(genParticleLabel,genParticles)
        
        func(i,
             event,
             *func_args,
             muons=muons,
             electrons=electrons,
             photons=photons,
             taus=taus,
             jets=jets,
             mets=mets,
             vertices=vertices,
             beamspot=beamspot,
             genParticles=genParticles,
             **func_kwargs)


In [ ]:
hppFiles = {
    200: [
        'root://cmsxrootd.hep.wisc.edu//store/user/dntaylor/2016-03-19_DevTools_76X_v1/HPlusPlusHMinusMinusHTo4L_M-200_13TeV-pythia8/2016-03-19_DevTools_76X_v1/160320_073817/0000/miniTree_1.root',
        'root://cmsxrootd.hep.wisc.edu//store/user/dntaylor/2016-03-19_DevTools_76X_v1/HPlusPlusHMinusMinusHTo4L_M-200_13TeV-pythia8/2016-03-19_DevTools_76X_v1/160320_073817/0000/miniTree_2.root',
        'root://cmsxrootd.hep.wisc.edu//store/user/dntaylor/2016-03-19_DevTools_76X_v1/HPlusPlusHMinusMinusHTo4L_M-200_13TeV-pythia8/2016-03-19_DevTools_76X_v1/160320_073817/0000/miniTree_3.root',
        'root://cmsxrootd.hep.wisc.edu//store/user/dntaylor/2016-03-19_DevTools_76X_v1/HPlusPlusHMinusMinusHTo4L_M-200_13TeV-pythia8/2016-03-19_DevTools_76X_v1/160320_073817/0000/miniTree_4.root',
        'root://cmsxrootd.hep.wisc.edu//store/user/dntaylor/2016-03-19_DevTools_76X_v1/HPlusPlusHMinusMinusHTo4L_M-200_13TeV-pythia8/2016-03-19_DevTools_76X_v1/160320_073817/0000/miniTree_5.root',
        'root://cmsxrootd.hep.wisc.edu//store/user/dntaylor/2016-03-19_DevTools_76X_v1/HPlusPlusHMinusMinusHTo4L_M-200_13TeV-pythia8/2016-03-19_DevTools_76X_v1/160320_073817/0000/miniTree_6.root',
        'root://cmsxrootd.hep.wisc.edu//store/user/dntaylor/2016-03-19_DevTools_76X_v1/HPlusPlusHMinusMinusHTo4L_M-200_13TeV-pythia8/2016-03-19_DevTools_76X_v1/160320_073817/0000/miniTree_7.root',
        'root://cmsxrootd.hep.wisc.edu//store/user/dntaylor/2016-03-19_DevTools_76X_v1/HPlusPlusHMinusMinusHTo4L_M-200_13TeV-pythia8/2016-03-19_DevTools_76X_v1/160320_073817/0000/miniTree_8.root',
    ],
}

def findDecay(genParticles,m_pdgid,d1_pdgid,d2_pdgid):
    for i,genParticle in enumerate(genParticles.product()):
        if genParticle.pdgId()==m_pdgid:
            if genParticle.numberofDaughters()>1:
                d1 = genParticle.daughter(0).pdgId()
                d2 = genParticle.daughter(1).pdgId()
                if (d1==d1_pdgid and d2==d2_pdgid) or (d2==d1_pdgid and d1==d2_pdgid):
                    return True
    return False

higgsChannels = ['ee','em','et','mm','mt','tt']
pdgMap = {
    11: 'e',
    13: 'm',
    15: 't',
}


def fill(i,event,decayMap,**kwargs):
    genParticles = kwargs.pop('genParticles')
    
    chanString = ''
    for s in [1,-1]:
        h = -1*s*9900041                     # h++ in pythia8
        for l1 in [s*11, s*13, s*15]:        # lepton 1
            for l2 in [s*11, s*13, s*15]:    # lepton 2
                if abs(l2)<abs(l1): continue # skip double counting
                hasDecay = self.findDecay(genParticles,h,l1,l2)
                if hasDecay:
                    chanString += pdgMap[abs(l1)]
                    chanString += pdgMap[abs(l2)]
                    
    decayMap[chanString] += 1



        

#lview = client.load_balanced_view()
results = {}

for mass in hppFiles:
    results[mass] = {}
    for hpp in higgsChannels:
        for hmm in higgsChannels:
            results[mass][hpp+hmm] = 0
    for f in hppFiles[mass]:
        events = Events(f)
        process(events,fill,func_args=[results[mass]])

import json
print json.dumps(results,sort_keys=True,indent=4)